<a href="https://colab.research.google.com/github/aubricot/object_detection_for_image_cropping/blob/master/chiroptera_train_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training YOLO in Darkflow to detect bats (Chiroptera) from EOL images
---
*Last Updated 11 February 2020*   
Use images and annotation files to train YOLO in Darkflow to detect bats from EOL images.

Datasets exported from [preprocessing.ipynb](https://colab.research.google.com/github/aubricot/object_detection_for_image_cropping/blob/master/preprocessing.ipynb) were converted to xml formatted annotation files before use in this notebook. Images were already downloaded to Google Drive in preprocessing.ipynb. 

Annotations should be uploaded to Google Drive for use in this notebook after installing darkflow (under Installs below).

Exported detection results (json files) can be used to calculate model precision for comparison with Faster-RCNN and SSD models using [calculate_error_mAP.ipynb](https://colab.research.google.com/github/aubricot/object_detection_for_image_cropping/blob/master/calculate_error_mAP.ipynb). 

Notes:   
* For each 24 hour period on Google Colab, you have up to 12 hours of GPU access. Training the object detection model on bats took 30 hours split into 3 days.

* Make sure to set the runtime to Python 2 with GPU Hardware Accelerator.   

References:   
* [Official Darkflow training instructions](https://github.com/thtrieu/darkflow)   
* [Medium Blog on training using YOLO via Darkflow in Colab](https://medium.com/coinmonks/detecting-custom-objects-in-images-video-using-yolo-with-darkflow-1ff119fa002f)

## Installs
---

In [0]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
# Change to your working directory
%cd drive/My Drive/fall19_smithsonian_informatics/train

# Install libraries
# Make sure you are using Python 3.6
!python --version
!pip install tensorflow-gpu==1.15.0rc2
!pip install cython
!pip install opencv-python

In [0]:
# Download darkflow (the tensorflow implementation of YOLO)
import os
import pathlib
import shutil 

if os.path.exists("darkflow-master"):
  %cd darkflow-master/darkflow
  !pwd

elif not os.path.exists("darkflow-master"):
    !git clone --depth 1 https://github.com/thtrieu/darkflow.git
    # Compile darkflow
    %cd darkflow
    !python setup.py build_ext --inplace
    # Rename darkflow to darkflow-master to distinguish between folder names
    shutil.move('/content/drive/My Drive/fall19_smithsonian_informatics/train/darkflow', 
              '/content/drive/My Drive/fall19_smithsonian_informatics/train/darkflow-master')

# Change wd to darkflow-master
%cd ../

#### Before proceeding to the next steps, you should manually upload annotations to your Google Drive. Test annotations should be uploaded to train/test_ann. Train annotations should be uploaded to darkflow-master/test/training/annotations. After uploading, return to this notebook and click refresh in the file browser on the left.

### Imports   
---

In [0]:
%cd darkflow-master

# For importing/exporting files, working with arrays, etc
from google.colab import files
import os
import pathlib
import imageio
import time
import csv
import urllib
import numpy as np
import pandas as pd

# For the actual object detection
from darkflow.net.build import TFNet

# For drawing onto and plotting the images
import matplotlib.pyplot as plt
import cv2
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

### Model Preparation (only need to run these once)
---   
For detailed instructions on training YOLO using a custom dataset, see the [Darkflow GitHub Repository](https://github.com/thtrieu/darkflow).

In [0]:
# Test installation, you should see an output with different parameters for flow
!python flow --h

In [0]:
# Upload yolo.weights, pre-trained weights file (for YOLO v2) from Google drive 
weights = 'bin/yolo'
weights_file = weights + '.weights'
if not os.path.exists('weights_file'): 
  !gdown --id 0B1tW_VtY7oniTnBYYWdqSHNGSUU
  !mkdir bin
  !mv yolo.weights bin

# Make new label file/overwrite existing labels.txt downloaded with darkflow
!echo "Chiroptera" > labels.txt

# Download model config file edited for training darkflow to identify bats (yolo-1c = yolo to identify 1 class)
mod_config = 'cfg/yolo-1c'
mod_config_file = config + '.cfg'
if not os.path.exists('mod_config_file'):
  %cd cfg
  !gdown --id 1bjt5Mqvf4AZSLNARgtgmZsfHZSyFj2yx
  %cd ../

## Train the model
---

In [0]:
# List different parameters for flow
!python flow --h

In [0]:
# Train model (yolo-1c.cfg) using pre-trained weights from basal layers of yolo.weights, the top layer will be trained from scracth to detect Lepidoptera
# Change the dataset and annotation directories to your paths in Google Drive
%cd darkflow-master
!python flow --model cfg/yolo-1c.cfg --train --trainer adam --load bin/yolo.weights --gpu 0.8 --epoch 3000 --dataset "/content/drive/My Drive/fall19_smithsonian_informatics/train/images" --annotation "test/training/annotations" --savepb

In [0]:
# Resume training from last checkpoint
!python flow --load -1 --model cfg/yolo-1c.cfg --train --savepb --trainer adam --gpu 0.8 --epoch 3000 --dataset "/content/drive/My Drive/fall19_smithsonian_informatics/train/images" --annotation "test/training/annotations"

In [0]:
# Save the last checkpoint to protobuf file
!python flow --model cfg/yolo-1c.cfg --load -1 --savepb

In [0]:
# Resume training from protobuf file
!python flow --load -1 --pbLoad built_graph/yolo-1c.pb --metaLoad built_graph/yolo-1c.meta --train --savepb --trainer adam --gpu 0.8 --epoch 3000 --dataset "/content/drive/My Drive/fall19_smithsonian_informatics/train/images" --annotation "test/training/annotations"

### When finished training

In [0]:
# Export detection results as json files fo calculating mAP (mean average precision, a performance measure to compare models) using calculate_error_mAP.ipynb
!python flow --pbLoad built_graph/yolo-1c.pb --gpu 0.8 --metaLoad built_graph/yolo-1c.meta --imgdir "/content/drive/My Drive/fall19_smithsonian_informatics/train/test_images" --json

In [0]:
# Optional: if want to run test images through detector AND SAVE OUTPUT IMAGES with detection boxes in test_images/out
# If you want to only view detection boxes on images and not save images with detection boxes, go to "Run test images" below
!python flow --pbLoad built_graph/yolo-1c.pb --gpu 0.8 --metaLoad built_graph/yolo-1c.meta --imgdir "/content/drive/My Drive/fall19_smithsonian_informatics/train/test_images"

## Run test images through the trained object detector
---
Test image detection boxes are only needed for calculating mAP (mean average precision, a performance measure to compare models) and not for cropping. The functions below will only display resulting detection boxes on test images for visualization, but does not save their coordinates to a spreadsheet. 

### Prepare object detection functions and settings

In [0]:
# For loading images into computer-readable format
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

# Function for loading images from urls
def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  return image

# For drawing bounding boxes around detected objects on images
def boxing(image, predictions):
    newImage = np.copy(image)
    im_height, im_width, im_depth = image.shape

    # Organize results of object detection for plotting and export
    for result in predictions:
        xmin = result['topleft']['x']
        ymin = result['topleft']['y']

        xmax = result['bottomright']['x']
        ymax = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        # Only show boxes that are above set confidence and for the label Chiroptera
        # Optional: change confidence and label values
        if confidence > 0 and result['label'] == 'Chiroptera' :
            # Draw boxes on images
            fontScale = min(im_width,im_height)/(600)
            newImage = cv2.rectangle(newImage, (xmin, ymax), (xmax, ymin), (255, 0, 157), 3)
            newImage = cv2.putText(newImage, label, (xmin, ymax-5), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (153, 255, 255), 5, cv2.LINE_AA)
    return newImage

# Define parameters for "flow"ing the images through the model
# Optional: adjust detection confidence threshold
params = {
    'model': 'cfg/yolo-1c.cfg',
    'load': 'bin/yolo.weights',
    'gpu': 0.8,
    #'threshold': 0.1, 
    'pbLoad': 'built_graph/yolo-1c.pb', 
    'metaLoad': 'built_graph/yolo-1c.meta' 
}

# Run the model
tfnet = TFNet(params)

### Run test images through object detector

In [0]:
# Test trained model on test images
from PIL import Image

# Update path to your test images
PATH_TO_TEST_IMAGES_DIR = '/content/drive/My Drive/fall19_smithsonian_informatics/train/test_images'
names = os.listdir(PATH_TO_TEST_IMAGES_DIR)
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, name) for name in names]

# Loops through first 5 image urls from the text file
for im_num, im_path in enumerate(TEST_IMAGE_PATHS[:5], start=1):

    # Load in image
    image = Image.open(im_path)
    image_np = load_image_into_numpy_array(image)
    # Record inference time
    start_time = time.time()
    # Detection
    result = tfnet.return_predict(image_np)
    end_time = time.time()
    # Draw boxes on image
    boxing(image_np, result)
    # Display progress message after each image
    print('Detection complete in {} of 145 test images'.format(im_num))

    # Plot and show detection boxes on images
    # If running detection on >50 images, comment out this portion
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(boxing(image_np, result))
    plt.title('{}) Inference time: {}'.format(im_num, format(end_time-start_time, '.2f')))

### Run other images (from individual URLs) through object detector

In [0]:
# Test trained model on test images
from PIL import Image

# Put your urls here
image_urls = ["https://upload.wikimedia.org/wikipedia/commons/b/be/Batman_%28retouched%29.jpg",
              "https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/Bela_Lugosi_as_Dracula%2C_anonymous_photograph_from_1931%2C_Universal_Studios.jpg/690px-Bela_Lugosi_as_Dracula%2C_anonymous_photograph_from_1931%2C_Universal_Studios.jpg"]

# Loops through image_urls
for im_num, image_url in enumerate(image_urls, start=1):
  try:
    # Load in image
    image_np = url_to_image(image_url)
    # Record inference time
    start_time = time.time()
    # Detection
    result = tfnet.return_predict(image_np)
    end_time = time.time()
    # Draw boxes on image
    boxing(image_np, result)
    # Display progress message after each image
    print('Detection complete in {} of 2 test images'.format(im_num))

    # Plot and show detection boxes on images
    # If running detection on >50 images, comment out this portion
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(boxing(image_np, result))
    plt.title('{}) Inference time: {}'.format(im_num, format(end_time-start_time, '.2f')))

  except:
    print('Check if URL from {} is valid'.format(image_url))

## Run EOL image bundles through the trained object detector & save results for cropping
---
Display resulting detection boxes on images and save their coordinates to lepidoptera_det_crops_yolo.tsv for use cropping EOL images.

### Prepare object detection functions and settings

In [0]:
# For loading images into computer-readable format
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

# Function for loading images from urls
def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  return image

# For drawing bounding boxes around detected objects on images
def boxing(image, predictions):
    newImage = np.copy(image)
    im_height, im_width, im_depth = image.shape

    # Organize results of object detection for plotting and export
    for result in predictions:
        xmin = result['topleft']['x']
        ymin = result['topleft']['y']

        xmax = result['bottomright']['x']
        ymax = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        # Only show boxes that are above set confidence and for the label Chiroptera
        if confidence > 0 and result['label'] == 'Chiroptera' :
            # Draw boxes on images
            fontScale = min(im_width,im_height)/(600)
            newImage = cv2.rectangle(newImage, (xmin, ymax), (xmax, ymin), (255, 0, 157), 3)
            newImage = cv2.putText(newImage, label, (xmin, ymax-5), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (153, 255, 255), 5, cv2.LINE_AA)

            # Export detection results to det_crops_yolo.tsv
            # Change filename here if using 1000 or 20000 images dataset
            with open('/content/drive/My Drive/fall19_smithsonian_informatics/chiroptera_det_crops_yolo_1000.tsv', 'a') as out_file:
                  tsv_writer = csv.writer(out_file, delimiter='\t')
                  tsv_writer.writerow([image_url, im_height, im_width, 
                            xmin, ymin, xmax, ymax])

    return newImage

# Define parameters for "flow"ing the images through the model
# Optional: adjust detection confidence threshold
params = {
    'model': 'cfg/yolo-1c.cfg',
    'load': 'bin/yolo.weights',
    'gpu': 0.8,
    #'threshold': 0.1, 
    'pbLoad': 'built_graph/yolo-1c.pb', 
    'metaLoad': 'built_graph/yolo-1c.meta' 
}

# Run the model
tfnet = TFNet(params)

### Run images (from EOL image URL bundles) through object detector

In [0]:
# Use URLs from EOL image URL bundles
# Comment out to use either 1000 or 20000 image bundles
# 1000 Lepidoptera images
urls = 'https://editors.eol.org/other_files/bundle_images/files/images_for_Chiroptera_breakdown_download_000001.txt'
# 20000 Lepidoptera images
#urls = 'https://editors.eol.org/other_files/bundle_images/files/images_for_Chiroptera_20K_breakdown_download_000001.txt'

df = pd.read_csv(urls)
df.columns = ["link"]
pd.DataFrame.head(df)

In [0]:
# Write header row of output crops file
# For 1000 or 20000 image datasets, change filename here and in "Prepare object detection functions and settings -> def boxing -> Export detection results" above
with open('/content/drive/My Drive/fall19_smithsonian_informatics/train/chiroptera_det_crops_1000.tsv', 'a') as out_file:
                  tsv_writer = csv.writer(out_file, delimiter='\t')
                  tsv_writer.writerow(["image_url", "im_height", "im_width", 
                            "xmin", "ymin", "xmax", "ymax"])

In [0]:
# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

# Loops through first 5 image urls from the text file
for i, row in df.head(5).itertuples(index=True, name='Pandas'):

# For ranges of rows or all rows, use df.iloc
# Can be useful if running detection in batches
#for i, row in df.iloc[500:800].iterrows():

  try:
    # Record inference time
    start_time = time.time()
    # Load in image
    image_url = df1.get_value(i, "link")
    image = url_to_image(image_url)
    # Detection
    result = tfnet.return_predict(image_np)
    end_time = time.time()
    # Draw boxes on image
    boxing(image_np, result)
    # Display progress message after each image
    print('Detection complete in {} of 1000 test images'.format(im_num))

    # Plot and show detection boxes on images
    # If running detection on >50 images, comment out this portion
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(boxing(image_np, result))
    plt.title('{}) Inference time: {}'.format(i+1, format(end_time-start_time, '.2f')))

  except:
    print('Check if URL from {} is valid'.format(image_url))